In [1]:
# This notebook takes the train and test NumPy files from the saved location, train a model on the training data, and calculate the overall
# and class-wise accuracies of the model.

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.5.0


In [3]:
# checks if any GPU is present for this notebook or not-

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
# essentially checks the GPU type

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12362792289518130872, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14509932544
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13135363487127064120
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [5]:
# import all the necessary packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import itertools
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)
%matplotlib inline

In [6]:
# mount the google drive to load the necessary files (in the next cell).

from google.colab import drive
drive.mount('/content/Pdrive')

Mounted at /content/Pdrive


In [7]:
# Load the train and test data from the NumPy files saved from the first notebook.

X_train = np.load('/content/Pdrive/MyDrive/Colab Notebooks/XTrain.npy')

X_test = np.load('/content/Pdrive/MyDrive/Colab Notebooks/XTest.npy')

y_train = np.load('/content/Pdrive/MyDrive/Colab Notebooks/yTrain.npy')

y_test = np.load('/content/Pdrive/MyDrive/Colab Notebooks/yTest.npy')

In [8]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(26487, 36, 1)
(8828, 36, 1)
(26487,)
(8828,)


In [21]:
# Import all the necessary things for model-training.

import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dropout, Dense, Activation
from tensorflow.keras.layers import GlobalAveragePooling1D, BatchNormalization, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.regularizers import l1, l2, l1_l2 

In [22]:
    reg2 = l2(0.01)  # L2 regularizer

    model = Sequential()
    model.add(Conv1D(filters=128,kernel_size=1, activation="relu", input_shape = (36, 1)))
    model.add(Conv1D(filters=128,kernel_size=1, activation="relu"))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Dropout(0.1))
    model.add(Conv1D(filters=128, kernel_size=1, activation="relu"))
    model.add(Conv1D(filters=128, kernel_size=1, activation="relu"))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Dropout(0.1))
    model.add(Conv1D(filters=256, kernel_size=1, activation="relu"))
    model.add(Conv1D(filters=256, kernel_size=1, activation="relu"))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Dropout(0.1))
    model.add(Conv1D(filters=256, kernel_size=1, activation="relu"))
    model.add(Conv1D(filters=256, kernel_size=1, activation="relu"))
    model.add(MaxPooling1D(pool_size = 1))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(Flatten())
    model.add(Dense(units=256,activation="relu", kernel_regularizer = reg2, bias_regularizer = reg2))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(units=256,activation="relu", kernel_regularizer = reg2, bias_regularizer = reg2))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(units=10, activation="softmax"))
    

In [23]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_16 (Conv1D)           (None, 36, 128)           256       
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 36, 128)           16512     
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 36, 128)           0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 36, 128)           0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 36, 128)           16512     
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 36, 128)           16512     
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 36, 128)          

In [24]:
model.compile(optimizer = Adam(learning_rate = 0.0001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

model.fit(x = X_train, y = y_train, batch_size = 128, epochs = 60)

Epoch 1/60
207/207 [==============================] - 17s 43ms/step - loss: 7.8443 - accuracy: 0.3050
Epoch 2/60
207/207 [==============================] - 9s 43ms/step - loss: 5.4111 - accuracy: 0.4825
Epoch 3/60
207/207 [==============================] - 9s 43ms/step - loss: 4.0168 - accuracy: 0.5862
Epoch 4/60
207/207 [==============================] - 9s 43ms/step - loss: 3.1564 - accuracy: 0.6435
Epoch 5/60
207/207 [==============================] - 9s 44ms/step - loss: 2.5590 - accuracy: 0.6928
Epoch 6/60
207/207 [==============================] - 9s 45ms/step - loss: 2.1423 - accuracy: 0.7277
Epoch 7/60
207/207 [==============================] - 9s 45ms/step - loss: 1.8402 - accuracy: 0.7556
Epoch 8/60
207/207 [==============================] - 9s 46ms/step - loss: 1.6149 - accuracy: 0.7759
Epoch 9/60
207/207 [==============================] - 9s 46ms/step - loss: 1.4345 - accuracy: 0.7947
Epoch 10/60
207/207 [==============================] - 9s 45ms/step - loss: 1.2917 - accur

In [25]:
score = model.evaluate(X_test, y_test, batch_size = 128)

69/69 [==============================] - 3s 18ms/step - loss: 0.3345 - accuracy: 0.9572


In [26]:
# THE FINAL ACCURACY RESULT OF THE MODEL:

print("Accuracy: ", round(score[1]*100, 2))

Accuracy:  95.72


In [27]:
# y_pred = an array of all the predicted classes or labels.

y_pred = np.argmax(model.predict(X_test), axis = -1)

In [28]:
# TO FIND THE ACCURACY OF MODEL ON EACH OF THE CLASSES 

from sklearn.metrics import confusion_matrix

cmat = confusion_matrix(y_test, y_pred)    

all_acc = cmat.diagonal()/cmat.sum(axis = 1)    # all_acc is an array containing accuracies of the model on each of the classes 

In [29]:
all_acc

array([1.        , 0.99888889, 0.89437428, 0.99662542, 0.86538462,
       1.        , 1.        , 1.        , 0.81664792, 1.        ])

In [30]:
#PRINT THE APPROXIMATE VALUES OF ACCURACY OF MODEL ON EACH OF THE CLASSES:

for i in range(len(all_acc)):
  
  if(i==0):
    temp = 'ALMI'
  elif(i==1):
    temp = 'AMI'
  elif(i==2):
    temp = 'ASMI'
  elif(i==3):
    temp = 'ILMI'
  elif(i==4):
    temp = 'IMI'
  elif(i==5):
    temp = 'IPLMI'
  elif(i==6):
    temp = 'IPMI'
  elif(i==7):
    temp = 'LMI'
  elif(i==8):
    temp = 'NORM'
  else:
    temp = 'PMI'
  
  print('Accuracy of the model on   ' + temp + ' = ', np.round(all_acc[i]*100, 2))

Accuracy of the model on   ALMI =  100.0
Accuracy of the model on   AMI =  99.89
Accuracy of the model on   ASMI =  89.44
Accuracy of the model on   ILMI =  99.66
Accuracy of the model on   IMI =  86.54
Accuracy of the model on   IPLMI =  100.0
Accuracy of the model on   IPMI =  100.0
Accuracy of the model on   LMI =  100.0
Accuracy of the model on   NORM =  81.66
Accuracy of the model on   PMI =  100.0
